# » `Dependencias`:

In [ ]:
# !pip install plotly
# !pip install sqlalchemy
# !pip install numpy
# !pip install matplotlib

In [67]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import re
import matplotlib.pyplot as plt
# import cufflinks as cf
# from IPython.display import display, HTML

# cf.set_config_file

## 1. Conección → SIM(172.27.0.124)

In [2]:
# spring.datasource.url = jdbc:sqlserver://172.27.250.27;databaseName=SIRIM
SERVER = '172.27.0.124' # '172.27.0.242'
#DRIVER = 'SQL Server Native Client 11.0'
DRIVER = 'ODBC Driver 17 for SQL Server'
DATABASE = 'SIM'
USERNAME = 'userestadistica' # 'udesa'
PASSWORD = '$Us3R_3sT4d1sTic4$' # 'DESARROLLO2006'
DATABASE_CONNECTION = f'mssql://{USERNAME}:{PASSWORD}@{SERVER}/{DATABASE}?driver={DRIVER}'

engine = create_engine(DATABASE_CONNECTION)
connection = engine.connect()

## 2. Métodos genérico:

In [3]:
def get_query_sql(query):
  try:
    df = pd.read_sql(query, connection)
    return df
  except:
    print('¡Ocurrió un error!')

## 3. Visualización de datos: `INF-JUL2024`

### 3.1 1. Se define como regla, que las e-gates no podrán ser usadas por ciudadanos `PERUANOS` con documentos de viaje `DNI`.

In [4]:
# Extracción
df_1 = pd.read_excel('./anexos/A1.xlsx')

In [246]:
# Campos: ['Id Persona', 'Id Mov Migratorio','Tipo Movimiento', 'Fecha Movimiento']
df_1_1 = df_1.groupby(by=['Tipo Movimiento']).agg({ 'Id Mov Migratorio': [np.ma.count] }).droplevel(level=0, axis=1)
len(df_1)

#df_1_f = df_1_1.plot.bar(legend=False, xlabel='')

25

### 3.2 2. Se define como regla, que los trámites de `Cambio de Clase de Visa` Aprobados, deben tener sus etapas `Finalizadas`.

In [75]:
df_2 = pd.read_excel('./anexos/A2.xlsx')

In [248]:
# ['Número Trámite', 'Fecha Trámite']

df_2['Año Trámite'] =  pd.to_datetime(df_2['Fecha Trámite']).dt.year
df_2_1 = df_2.groupby(by=['Año Trámite']).agg({ 'Número Trámite': [np.ma.count] }).droplevel(level=0, axis=1)

len(df_2)
""" df_2_f = df_2_1.plot(legend=False, xlabel='')
df_2_f.set_xticks(ticks=[2022, 2023, 2024]) """

14

### 3.3 3. Se define como regla, que las `Solicitud de Cambio Calidad Migratoria` aprobadas, deben registrar una fecha de vencimiento.

In [137]:
df_3 = pd.read_excel('./anexos/A3.xlsx')

In [250]:
# ['Número Trámite', 'dFechaAprobacion']
df_3['Año Aprobacion'] = df_3['dFechaAprobacion'].dt.year

len(df_3)

""" df_3_1 = df_3.groupby(by=['Año Aprobacion']).agg({ 'Número Trámite': 'count' }) \
                                                              
df_3_f = df_3_1.plot(legend=False, xlabel='') """
         

186

### 3.4 4. Se define como regla, que los Carnet de Extranjería, deben tener una vigencia de 3 años en el caso de menores de edad.

In [152]:
df_4 = pd.read_excel('./anexos/A4.xlsx')

In [251]:
import math
# ['Número Carnet', 'Fecha Emisión', 'Fecha Caducidad']
df_4['Días Exceso((C - E) - 3)'] = ((df_4['Fecha Caducidad'] - df_4['Fecha Emisión']).dt.days - (3 * 365))

df_4_1 = df_4.groupby(by=['Días Exceso((C - E) - 3)']).agg({ 'Número Carnet': 'count' })
df_4_f = df_4_1.plot(legend=False, xlabel='Días', ylabel='C.E. Menores')


64651

### 3.5 5. Se define como regla, que la población extranjera menor de edad, no debe registrar una calidad migratoria de: `OFICIAL, ARTISTA, TRIPULANTE, NEGOCIOS, TRABAJADOR y DIPLOMATICA`.

In [206]:
df_5 = pd.read_excel('./anexos/A5.xlsx')

In [252]:
# ['Id Persona', 'Nombres', 'Apellido 1', 'Apellido 2', 'Sexo', 'Fecha Nacimiento', 'Nacionalidad', 'Fecha Ingreso', 'Edad', 'Calidad Migratoria']

df_5_1 = df_5.groupby(by=['Calidad Migratoria']).agg({ 'Id Persona': 'count' })
df_5_f = df_5_1.plot.bar(legend=False, xlabel='')


501

In [47]:
month = 5
year = 2024
path = f'{year}-{str(month).zfill(2)}'
path

'2024-05'